In [1]:
import nltk

nltk.download("punkt")
from nltk import word_tokenize
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicho\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# reading in the data
train_data = pd.read_csv("WikiQA-train.tsv", sep="\t")
test_data = pd.read_csv("WikiQA-test.tsv", sep="\t")

Extract the unique questions from the train and test data frames, including the documentID and the DocumentTitle


In [3]:
def get_questions_documenttag(data):
    qd = data[
        ["Question", "QuestionID", "DocumentID", "DocumentTitle"]
    ].drop_duplicates()
    return qd


train_question_doctag = get_questions_documenttag(train_data)
test_question_doctag = get_questions_documenttag(test_data)

In [4]:
# get unique questions
train_questions = train_question_doctag["Question"]
test_questions = test_question_doctag["Question"]

In [5]:
# get the unique document ids
train_docid = train_question_doctag["DocumentID"]
test_docid = test_question_doctag["DocumentID"]

Extract the answers to those questions.


In [6]:
def get_answers(data, questions, documentids):
    answers = []  # list of answers
    for q in range(len(questions)):
        question = questions.iloc[q]
        doc_id = documentids.iloc[q]  # add the document id
        df = data[data["Question"] == question]
        index = df.loc[df["Label"] == 1]["Sentence"].index.values
        if len(index) == 0:  # if no answer found
            answers.append([question, doc_id, "No answer"])
        else:  # if 1 answer found
            answers.append([question, doc_id, df.loc[index[0], "Sentence"]])
    return answers


train_answers = pd.DataFrame(get_answers(train_data, train_questions, train_docid))
test_answers = pd.DataFrame(get_answers(test_data, test_questions, test_docid))

The above get_answers returns train_answers and test_answers which, gives us in the following columns

-   Question
-   Related Document ID
-   Answer (if no answer to that question, return no answer)


In [7]:
def get_documents(data, questions, documentids):  # (done by Finn, tweaked by Dan)
    documents = []
    for q in range(len(questions)):
        question = questions.iloc[q]
        doc_id = documentids.iloc[q]  # add the document id
        df = data[data["Question"] == question]
        sentences = df["Sentence"].tolist()
        for i in range(0, len(sentences) - 1):
            sentences[i] = sentences[i] + " "
        documents.append([doc_id, "".join(sentences)])
    return documents


train_documents = pd.DataFrame(
    get_documents(train_data, train_questions, train_docid)
)  # return the individual document in list
test_documents = pd.DataFrame(
    get_documents(test_data, test_questions, test_docid)
)  # return the individual document in list

The above train_documents and test_documents called from the get_documents gives us in the following columns

-   Document ID
-   Full Document


In [8]:
# renaming all the columns for more standardised access
train_answers.columns = ["Question", "DocumentID", "Answer"]
test_answers.columns = ["Question", "DocumentID", "Answer"]
train_documents.columns = ["DocumentID", "Document"]
test_documents.columns = ["DocumentID", "Document"]

In [9]:
# result is 2117, 2117, 630, 630

len(train_answers), len(train_documents), len(test_answers), len(test_documents)

(2117, 2117, 630, 630)

**Prior to tagging, we should maybe clean the document and answers first:** (stopped here)

Maybe?

-   lowercase (might lose context, but we can use on questions)
-   removing any punctuation or weird symbols (do)
-   removal of stop words? (probably not)

Make sure that the pre-processing is standardised to be the same throughout doc and ans.


In [10]:
def preprocess_lower(text):
    # Lowercase the text for question, answer and documents
    text = text.lower()
    pattern = r"[^a-zA-Z0-9\s]"
    cleaned_text = re.sub(pattern, " ", text)
    return cleaned_text


train_answers[["Question", "Answer"]] = train_answers[["Question", "Answer"]].applymap(
    preprocess_lower
)
train_documents["Document"] = train_documents["Document"].apply(preprocess_lower)
test_answers[["Question", "Answer"]] = test_answers[["Question", "Answer"]].applymap(
    preprocess_lower
)
test_documents["Document"] = test_documents["Document"].apply(preprocess_lower)

In [11]:
train_documents

,DocumentID,Document
0,D1,a partly submerged glacier cave on perito more...
1,D2,in physics circular motion is a movement of ...
2,D5,apollo creed is a fictional character from the...
3,D6,in the united states the title of federal jud...
4,D7,the beretta 21a bobcat is a small pocket sized...
...,...,...
2112,D2805,blue mountain state is an american comedy seri...
2113,D2806,apple inc formerly apple computer inc is ...
2114,D2807,section 8 housing in the south bronx section 8...
2115,D2808,restaurants categorized by type and informatio...


In [12]:
def labelling(documents, answers):
    tagged_documents = []
    for q in range(len(answers)):
        tagged_document = []
        qn = answers["Question"].loc[q]
        doc_id = answers["DocumentID"].loc[q]
        content = documents.loc[documents["DocumentID"] == doc_id, "Document"].values[0]
        answer = answers["Answer"].loc[q]

        if answer == "no answer":
            tokens = word_tokenize(content)
            for j in range(len(tokens)):
                tagged_document.append("N")  # none
        else:
            parts = content.partition(answer)
            for j in range(len(parts)):
                tokens = word_tokenize(parts[j])
                if j == 1:
                    tagged_document.append("S")  # start of answer
                    for k in range(len(tokens) - 2):
                        tagged_document.append("I")  # inside of answer
                    tagged_document.append("E")  # end of answer
                else:
                    for k in range(len(tokens)):
                        tagged_document.append("N")  # outside answer
        tagged_documents.append(tagged_document)
    return tagged_documents


train_doc_ans_labels = labelling(train_documents, train_answers)
test_doc_ans_labels = labelling(test_documents, test_answers)

In [13]:
# check if tags are good
def testing_tokens(ind, labels, documents, answers):
    for i, j in zip(labels[ind], word_tokenize(documents["Document"][ind])):
        print([i, j])
    print(answers["Answer"][ind])


testing_tokens(100, train_doc_ans_labels, train_documents, train_answers)

['N', 'the']
['N', 'big']
['N', 'ten']
['N', 'conference']
['N', 'formerly']
['N', 'western']
['N', 'conference']
['N', 'and']
['N', 'big']
['N', 'nine']
['N', 'conference']
['N', 'is']
['N', 'the']
['N', 'oldest']
['N', 'division']
['N', 'i']
['N', 'college']
['N', 'athletic']
['N', 'conference']
['N', 'in']
['N', 'the']
['N', 'united']
['N', 'states']
['S', 'its']
['I', 'twelve']
['I', 'member']
['I', 'institutions']
['I', 'which']
['I', 'are']
['I', 'primarily']
['I', 'flagship']
['I', 'research']
['I', 'universities']
['I', 'in']
['I', 'their']
['I', 'respective']
['I', 'states']
['I', 'well']
['I', 'regarded']
['I', 'academically']
['I', 'and']
['I', 'with']
['I', 'relatively']
['I', 'large']
['I', 'student']
['I', 'enrollment']
['I', 'are']
['I', 'located']
['I', 'primarily']
['I', 'in']
['I', 'the']
['I', 'midwest']
['I', 'stretching']
['I', 'from']
['I', 'nebraska']
['I', 'in']
['I', 'the']
['I', 'west']
['I', 'to']
['I', 'penn']
['I', 'state']
['I', 'in']
['I', 'the']
['E', 'e

Cleaned Documents: train and test

train_answers - contains the ['Question','DocumentID','Answer']

train_documents - contains the ['DocumentID','Document']

train_doc_ans_labels - contains a list of list of answer tags for each document,


In [14]:
# To prepare the document for word embeddings:
train_doc_ques = pd.DataFrame(
    {"Document": train_documents["Document"], "Question": train_answers["Question"]}
)
test_doc_ques = pd.DataFrame(
    {"Document": test_documents["Document"], "Question": test_answers["Question"]}
)

### Word Embeddings

To use the CBOW model, we need the data in sentences. Extract this from the original dataset, don't use sent_tokenise, will mess with some of the fullstops, we want to maintain structure from above


In [15]:
def word_tokens(data):
    sentence_list = []
    for i in range(len(data)):
        sentence_list.append(word_tokenize(data[i]))
    return sentence_list


train_doc_list = word_tokens(train_doc_ques["Document"])
train_ques_list = word_tokens(train_doc_ques["Question"])
test_doc_list = word_tokens(test_doc_ques["Document"])
test_ques_list = word_tokens(test_doc_ques["Question"])

In [16]:
combined_text = train_doc_list + train_ques_list + test_doc_list + test_ques_list

In [17]:
# model trained, don't have to run this multiple times
wc_cbow_model = Word2Vec(
    sentences=combined_text,
    vector_size=100,
    window=5,
    min_count=1,
    workers=2,
    epochs=30,
)
wc_cbow_model.save("cbow.model")

To implement QA

1. Word Embeddings, using CBOW
2. Feature Extraction 1 - POS tags
3. Feature Extraction 2 - TF-IDF
4. Feature Extraction 3 - NER


In [18]:
def get_word_embeddings(doc):
    tokenized_doc = word_tokenize(doc)
    embeddings = [wc_cbow_model.wv[word] for word in tokenized_doc]
    return embeddings


train_doc_ques["Doc_Embeddings"] = train_doc_ques["Document"].apply(get_word_embeddings)
train_doc_ques["Q_Embeddings"] = train_doc_ques["Question"].apply(get_word_embeddings)
test_doc_ques["Doc_Embeddings"] = test_doc_ques["Document"].apply(get_word_embeddings)
test_doc_ques["Q_Embeddings"] = test_doc_ques["Question"].apply(get_word_embeddings)

In [19]:
train_doc_ques["Doc_Tokens"] = train_doc_ques["Document"].apply(word_tokenize)
train_doc_ques["Q_Tokens"] = train_doc_ques["Question"].apply(word_tokenize)
test_doc_ques["Doc_Tokens"] = test_doc_ques["Document"].apply(word_tokenize)
test_doc_ques["Q_Tokens"] = test_doc_ques["Question"].apply(word_tokenize)

In [20]:
def check_count(doc):
    count = 0
    for i in range(len(doc)):
        if len(doc["Doc_Embeddings"][i]) != len(doc["Doc_Tokens"][i]):
            count += 1
        elif len(doc["Q_Embeddings"][i]) != len(doc["Q_Tokens"][i]):
            count += 1
        else:
            continue
    return count


check_count(train_doc_ques)  # looks good

0

Note, need to convert the POS tags, NER tags into embeddings. After this, pad the questions and answers to the max question/document length in the combined training and test set.

### PoS Tagging


In [21]:
# Apply the pos tags to the tokens
from nltk.tag import pos_tag

# download the dependency and resource as required
nltk.download("averaged_perceptron_tagger")

train_doc_ques["Doc_POS"] = train_doc_ques["Doc_Tokens"].apply(pos_tag)
train_doc_ques["Q_POS"] = train_doc_ques["Q_Tokens"].apply(pos_tag)
test_doc_ques["Doc_POS"] = test_doc_ques["Doc_Tokens"].apply(pos_tag)
test_doc_ques["Q_POS"] = test_doc_ques["Q_Tokens"].apply(pos_tag)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nicho\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [22]:
# checking the POS tags: # looks ok
test_doc_ques["Q_POS"][0]

[('how', 'WRB'),
 ('african', 'JJ'),
 ('americans', 'NNS'),
 ('were', 'VBD'),
 ('immigrated', 'VBN'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('us', 'PRP')]

In [23]:
# Extract all unique POS Tags
all_pos_tags = (
    train_doc_ques["Doc_POS"].tolist()
    + test_doc_ques["Doc_POS"].tolist()
    + train_doc_ques["Q_POS"].tolist()
    + test_doc_ques["Q_POS"].tolist()
)


def get_unique_pos(data):
    pos_tags = set()
    for item in data:
        for _, pos_tag in item:
            pos_tags.add(pos_tag)

    pos_tag_index = {tag: i for i, tag in enumerate(sorted(pos_tags))}
    return pos_tag_index


pos_iden = get_unique_pos(all_pos_tags)  # list of tags
pos_iden

{'$': 0,
 'CC': 1,
 'CD': 2,
 'DT': 3,
 'EX': 4,
 'FW': 5,
 'IN': 6,
 'JJ': 7,
 'JJR': 8,
 'JJS': 9,
 'MD': 10,
 'NN': 11,
 'NNP': 12,
 'NNPS': 13,
 'NNS': 14,
 'PDT': 15,
 'POS': 16,
 'PRP': 17,
 'PRP$': 18,
 'RB': 19,
 'RBR': 20,
 'RBS': 21,
 'RP': 22,
 'SYM': 23,
 'TO': 24,
 'UH': 25,
 'VB': 26,
 'VBD': 27,
 'VBG': 28,
 'VBN': 29,
 'VBP': 30,
 'VBZ': 31,
 'WDT': 32,
 'WP': 33,
 'WP$': 34,
 'WRB': 35}

### NER Tagging


### Steps to run this:

-   pip install spacy
-   python -m spacy download en_core_web_sm

If loaded for the first time, restart kernel


In [24]:
# nltk using Spacy
# pip install -U spacy
!pip install -U spacy
!python -m spacy download en_core_web_sm
# python -m spacy download en_core_web_sm
import spacy
import en_core_web_sm

# loading pre-trained model of NER
nlp = en_core_web_sm.load()

     ---------------------------------------- 12.8/12.8 MB 5.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [25]:
def ner_tagging(texts):
    tagged_texts = []
    for text in texts:
        doc = spacy.tokens.Doc(nlp.vocab, words=text)
        nlp.get_pipe("ner")(doc)
        tagged_texts.append([(token.text, token.ent_type_) for token in doc])
    return tagged_texts

In [26]:
# Will take a while...
train_doc_ques["Doc_NER"] = ner_tagging(train_doc_ques["Doc_Tokens"])
train_doc_ques["Q_NER"] = ner_tagging(train_doc_ques["Q_Tokens"])
test_doc_ques["Doc_NER"] = ner_tagging(test_doc_ques["Doc_Tokens"])
test_doc_ques["Q_NER"] = ner_tagging(test_doc_ques["Q_Tokens"])

In [27]:
# Similar approach to the POS

# Extract all unique POS Tags
all_ner_tags = (
    train_doc_ques["Doc_NER"].tolist()
    + test_doc_ques["Doc_NER"].tolist()
    + train_doc_ques["Q_NER"].tolist()
    + test_doc_ques["Q_NER"].tolist()
)


def get_unique_ner(data):
    ner_tags = set()
    for item in data:
        for _, ner_tag in item:
            ner_tags.add(ner_tag)

    ner_tag_index = {tag: i for i, tag in enumerate(sorted(ner_tags))}
    return ner_tag_index


ner_iden = get_unique_pos(all_ner_tags)  # list of tags
ner_iden

{'': 0,
 'CARDINAL': 1,
 'DATE': 2,
 'EVENT': 3,
 'FAC': 4,
 'GPE': 5,
 'LANGUAGE': 6,
 'LAW': 7,
 'LOC': 8,
 'MONEY': 9,
 'NORP': 10,
 'ORDINAL': 11,
 'ORG': 12,
 'PERCENT': 13,
 'PERSON': 14,
 'PRODUCT': 15,
 'QUANTITY': 16,
 'TIME': 17,
 'WORK_OF_ART': 18}

In [28]:
# check ohv dims
ner_idx = ner_iden.values()
aa = np.eye(max(ner_idx) + 1)
# aa

### TF-IDF

First, calculate the document frequency of each token in the entire corpus (training documents + testing documents). The result is a dictionary where each token is a key and its value is the document frequency.


In [29]:
def document_frequency(corpus):
    """
    Computes the document frequency for every token in the corpus.
    Returns a dictionary {token: doc_freq, ...}
    """
    document_frequency = {}
    for document in corpus:
        for token in np.unique(document):
            try:
                document_frequency[token] += 1
            except:
                document_frequency[token] = 1
    return document_frequency


train_corpus = (
    train_doc_ques["Doc_Tokens"].tolist() + train_doc_ques["Q_Tokens"].tolist()
)
test_corpus = test_doc_ques["Doc_Tokens"].tolist() + test_doc_ques["Q_Tokens"].tolist()
train_doc_freq = document_frequency(train_corpus)
test_doc_freq = document_frequency(test_corpus)

Now calculate TF-IDF using the document frequency from above.


In [30]:
from collections import Counter
import math


def compute_tf_idf(corpus, doc_frequency):
    """
    Computes the term frequency inverse document frequency for every token in every document in the corpus.
    Returns a list the same shape as the list of tokenized documents except every token is replaced with the tf-idf
    for that token.
    """
    tf_idf = {}
    tf_idf_list = []
    N = len(doc_frequency)
    doc_id = 0
    for document in corpus:
        tf_idf_doc = []
        counter = Counter(document)
        total_num_words = len(document)
        for token in np.unique(document):
            tf = counter[token] / total_num_words
            df = doc_frequency[token]
            idf = math.log(N / (df + 1)) + 1
            tf_idf[doc_id, token] = tf * idf
        for token in document:
            tf_idf_doc.append(tf_idf[doc_id, token])
        tf_idf_list.append(tf_idf_doc)
        doc_id += 1
    return tf_idf_list


train_doc_ques["Doc_TFIDF"] = compute_tf_idf(
    train_doc_ques["Doc_Tokens"].tolist(), train_doc_freq
)
train_doc_ques["Q_TFIDF"] = compute_tf_idf(
    train_doc_ques["Q_Tokens"].tolist(), train_doc_freq
)
test_doc_ques["Doc_TFIDF"] = compute_tf_idf(
    test_doc_ques["Doc_Tokens"].tolist(), test_doc_freq
)
test_doc_ques["Q_TFIDF"] = compute_tf_idf(
    test_doc_ques["Q_Tokens"].tolist(), test_doc_freq
)

In [31]:
test_doc_ques

,Document,Question,Doc_Embeddings,Q_Embeddings,Doc_Tokens,Q_Tokens,Doc_POS,Q_POS,Doc_NER,Q_NER,Doc_TFIDF,Q_TFIDF
0,african immigration to the united states refer...,how african americans were immigrated to the us,"[[1.2212064, 0.0490168, -0.07282714, 0.9991929...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[african, immigration, to, the, united, states...","[how, african, americans, were, immigrated, to...","[(african, JJ), (immigration, NN), (to, TO), (...","[(how, WRB), (african, JJ), (americans, NNS), ...","[(african, ORG), (immigration, ORG), (to, ), (...","[(how, ), (african, NORP), (americans, NORP), ...","[0.2444438957631187, 0.16900818614783913, 0.28...","[0.7085526392283996, 0.934997901293929, 0.9907..."
1,a prison from old french prisoun also known...,how large were early jails,"[[0.66377705, -0.66919404, -2.8755727, 0.84947...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[a, prison, from, old, french, prisoun, also, ...","[how, large, were, early, jails]","[(a, DT), (prison, NN), (from, IN), (old, JJ),...","[(how, WRB), (large, JJ), (were, VBD), (early,...","[(a, ), (prison, ), (from, ), (old, ), (french...","[(how, ), (large, ), (were, ), (early, ), (jai...","[0.23736000397378015, 0.15055751080418858, 0.0...","[1.1336842227654393, 1.2988332831657334, 1.145..."
2,a small electrically powered pump a large el...,how a water pump works,"[[0.66377705, -0.66919404, -2.8755727, 0.84947...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[a, small, electrically, powered, pump, a, lar...","[how, a, water, pump, works]","[(a, DT), (small, JJ), (electrically, RB), (po...","[(how, WRB), (a, DT), (water, NN), (pump, NN),...","[(a, ), (small, ), (electrically, ), (powered,...","[(how, ), (a, ), (water, ), (pump, ), (works, )]","[0.18179077227423296, 0.07355471089668812, 0.1...","[1.1336842227654393, 0.8362375524614717, 1.412..."
3,lolita is a 1962 comedy drama film by stanley ...,how old was sue lyon when she made lolita,"[[-0.12167511, 0.10820423, 0.19476107, 0.15101...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[lolita, is, a, 1962, comedy, drama, film, by,...","[how, old, was, sue, lyon, when, she, made, lo...","[(lolita, NN), (is, VBZ), (a, DT), (1962, CD),...","[(how, WRB), (old, JJ), (was, VBD), (sue, NN),...","[(lolita, ), (is, ), (a, ), (1962, DATE), (com...","[(how, ), (old, ), (was, ), (sue, PERSON), (ly...","[0.22677748220200447, 0.031618095192546124, 0....","[0.6298245682030218, 0.7657996161610346, 0.528..."
4,each antibody binds to a specific antigen an...,how are antibodies used in,"[[0.42880726, 2.3034515, -4.884318, 3.6591587,...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[each, antibody, binds, to, a, specific, antig...","[how, are, antibodies, used, in]","[(each, DT), (antibody, NN), (binds, VBZ), (to...","[(how, WRB), (are, VBP), (antibodies, NNS), (u...","[(each, ), (antibody, ), (binds, ), (to, ), (a...","[(how, ), (are, ), (antibodies, ), (used, ), (...","[0.05405618374532114, 0.2406872267443911, 0.01...","[1.1336842227654393, 0.9670455719243073, 1.862..."
...,...,...,...,...,...,...,...,...,...,...,...,...
625,american cuts of beef including the brisket br...,where is the brisket from,"[[-2.170028, 1.3668909, -0.63760984, 0.5299978...","[[-0.8785929, -1.003242, 0.22002888, 1.2303052...","[american, cuts, of, beef, including, the, bri...","[where, is, the, brisket, from]","[(american, JJ), (cuts, NNS), (of, IN), (beef,...","[(where, WRB), (is, VBZ), (the, DT), (brisket,...","[(american, NORP), (cuts, ), (of, ), (beef, ),...","[(where, ), (is, ), (the, ), (brisket, ), (fro...","[0.044673596326364536, 0.15610157311465894, 0....","[1.1856348821507936, 0.8030996178906715, 0.792..."
626,the arm architecture describes a family of ris...,what is arm chipset,"[[-0.23086248, -0.8100985, -0.57038724, -0.738...","[[-1.2408428, 1.2921989, -2.3038976, -1.823878...","[the, arm, architecture, describes, a, family,...","[what, is, arm, chipset]","[(the, DT), (arm, NN), (archite

In [32]:
train_doc_ques

,Document,Question,Doc_Embeddings,Q_Embeddings,Doc_Tokens,Q_Tokens,Doc_POS,Q_POS,Doc_NER,Q_NER,Doc_TFIDF,Q_TFIDF
0,a partly submerged glacier cave on perito more...,how are glacier caves formed,"[[0.66377705, -0.66919404, -2.8755727, 0.84947...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[a, partly, submerged, glacier, cave, on, peri...","[how, are, glacier, caves, formed]","[(a, DT), (partly, RB), (submerged, VBN), (gla...","[(how, WRB), (are, VBP), (glacier, JJ), (caves...","[(a, ), (partly, ), (submerged, ), (glacier, )...","[(how, ), (are, ), (glacier, ), (caves, ), (fo...","[0.24679288919367473, 0.145118630440616, 0.161...","[1.0342546228402965, 0.8500722992276416, 1.989..."
1,in physics circular motion is a movement of ...,how are the directions of the velocity and for...,"[[-1.4634616, -0.11238795, -1.1756282, -0.3831...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[in, physics, circular, motion, is, a, movemen...","[how, are, the, directions, of, the, velocity,...","[(in, IN), (physics, NNS), (circular, JJ), (mo...","[(how, WRB), (are, VBP), (the, DT), (direction...","[(in, ), (physics, ), (circular, ), (motion, )...","[(how, ), (are, ), (the, ), (directions, ), (o...","[0.10158600494541081, 0.047978500911022494, 0....","[0.34475154094676547, 0.2833574330758805, 0.45..."
2,apollo creed is a fictional character from the...,how did apollo creed die,"[[-0.46987092, 0.51987296, 0.67675453, 0.11366...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[apollo, creed, is, a, fictional, character, f...","[how, did, apollo, creed, die]","[(apollo, NNS), (creed, VBP), (is, VBZ), (a, D...","[(how, WRB), (did, VBD), (apollo, VB), (creed,...","[(apollo, ORG), (creed, ), (is, ), (a, ), (fic...","[(how, ), (did, ), (apollo, ORG), (creed, ), (...","[0.1506184153725464, 0.2875274053249442, 0.057...","[1.0342546228402965, 1.1571023666681808, 1.787..."
3,in the united states the title of federal jud...,how long is the term for federal judges,"[[-1.4634616, -0.11238795, -1.1756282, -0.3831...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[in, the, united, states, the, title, of, fede...","[how, long, is, the, term, for, federal, judges]","[(in, IN), (the, DT), (united, JJ), (states, V...","[(how, WRB), (long, JJ), (is, VBZ), (the, DT),...","[(in, ), (the, GPE), (united, GPE), (states, G...","[(how, ), (long, ), (is, ), (the, ), (term, ),...","[0.08829203392786322, 0.3204713483897852, 0.16...","[0.6464091392751853, 0.7778250162349475, 0.429..."
4,the beretta 21a bobcat is a small pocket sized...,how a beretta model 21 pistols magazines works,"[[-0.23086248, -0.8100985, -0.57038724, -0.738...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[the, beretta, 21a, bobcat, is, a, small, pock...","[how, a, beretta, model, 21, pistols, magazine...","[(the, DT), (beretta, NN), (21a, CD), (bobcat,...","[(how, WRB), (a, DT), (beretta, NN), (model, N...","[(the, ), (beretta, ), (21a, ), (bobcat, ), (i...","[(how, ), (a, ), (beretta, PRODUCT), (model, )...","[0.21161559146390707, 0.8290244630930195, 0.22...","[0.6464091392751853, 0.4550243894508378, 1.243..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2112,blue mountain state is an american comedy seri...,where was blue mountain state filmed at,"[[-0.83070004, -0.042151436, 1.3289015, -1.111...","[[-0.8785929, -1.003242, 0.22002888, 1.2303052...","[blue, mountain, state, is, an, american, come...","[where, was, blue, mountain, state, filmed, at]","[(blue, JJ), (mountain, NN), (state, NN), (is,...","[(where, WRB), (was, VBD), (blue, JJ), (mounta...","[(blue, LOC), (mountain, LOC), (state, ), (is,...","[(where, ), (was, ), (blue, ), (mountain, ), (...","[0.29476307406437613, 0.3823599012669257, 0.22...","[0.7503384296942396, 0.5907013675064214, 1.063..."
2113,apple inc formerly apple computer inc is ...,when was apple computer founded,"[[-0.00645444, 0.5794082, 0.90777117, -0.26727...","[[-3.8799245, -2.42828, -1.0094839, 1.5448259,...","[apple, inc, formerly,

In [33]:
def one_hot_vectorize(
    pos_tagger, ner_tagger, data
):  # pass in the unique dict for ner or pos
    pos_idx = pos_tagger.values()
    pos_ohv = np.eye(max(pos_idx) + 1)  # create the ohv
    ner_idx = ner_tagger.values()
    ner_ohv = np.eye(max(ner_idx) + 1)

    dpos_full_ohv, dner_full_ohv = [], []  # lists to append to
    qpos_full_ohv, qner_full_ohv = [], []  # lists to append to

    for item in data["Doc_POS"]:
        sent_ohv = []
        for word in item:
            tag = word[1]
            pos_index_iden = pos_tagger[tag]
            sent_ohv.append(pos_ohv[pos_index_iden])
        dpos_full_ohv.append(sent_ohv)

    for item in data["Q_POS"]:
        sent_ohv = []
        for word in item:
            tag = word[1]
            pos_index_iden = pos_tagger[tag]
            sent_ohv.append(pos_ohv[pos_index_iden])
        qpos_full_ohv.append(sent_ohv)

    for item in data["Doc_NER"]:
        sent_ohv = []
        for word in item:
            tag = word[1]
            ner_index_iden = ner_tagger[tag]
            sent_ohv.append(ner_ohv[ner_index_iden])
        dner_full_ohv.append(sent_ohv)

    for item in data["Q_NER"]:
        sent_ohv = []
        for word in item:
            tag = word[1]
            ner_index_iden = ner_tagger[tag]
            sent_ohv.append(ner_ohv[ner_index_iden])
        qner_full_ohv.append(sent_ohv)

    return (dpos_full_ohv, qpos_full_ohv, dner_full_ohv, qner_full_ohv)

In [34]:
# get the ohv for doc
(
    train_doc_pos_ohv,
    train_q_pos_ohv,
    train_doc_ner_ohv,
    train_q_ner_ohv,
) = one_hot_vectorize(pos_iden, ner_iden, train_doc_ques)
test_doc_pos_ohv, test_q_pos_ohv, test_doc_ner_ohv, test_q_ner_ohv = one_hot_vectorize(
    pos_iden, ner_iden, test_doc_ques
)

In [35]:
train_doc_ques[:5]

,Document,Question,Doc_Embeddings,Q_Embeddings,Doc_Tokens,Q_Tokens,Doc_POS,Q_POS,Doc_NER,Q_NER,Doc_TFIDF,Q_TFIDF
0,a partly submerged glacier cave on perito more...,how are glacier caves formed,"[[0.66377705, -0.66919404, -2.8755727, 0.84947...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[a, partly, submerged, glacier, cave, on, peri...","[how, are, glacier, caves, formed]","[(a, DT), (partly, RB), (submerged, VBN), (gla...","[(how, WRB), (are, VBP), (glacier, JJ), (caves...","[(a, ), (partly, ), (submerged, ), (glacier, )...","[(how, ), (are, ), (glacier, ), (caves, ), (fo...","[0.24679288919367473, 0.145118630440616, 0.161...","[1.0342546228402965, 0.8500722992276416, 1.989..."
1,in physics circular motion is a movement of ...,how are the directions of the velocity and for...,"[[-1.4634616, -0.11238795, -1.1756282, -0.3831...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[in, physics, circular, motion, is, a, movemen...","[how, are, the, directions, of, the, velocity,...","[(in, IN), (physics, NNS), (circular, JJ), (mo...","[(how, WRB), (are, VBP), (the, DT), (direction...","[(in, ), (physics, ), (circular, ), (motion, )...","[(how, ), (are, ), (the, ), (directions, ), (o...","[0.10158600494541081, 0.047978500911022494, 0....","[0.34475154094676547, 0.2833574330758805, 0.45..."
2,apollo creed is a fictional character from the...,how did apollo creed die,"[[-0.46987092, 0.51987296, 0.67675453, 0.11366...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[apollo, creed, is, a, fictional, character, f...","[how, did, apollo, creed, die]","[(apollo, NNS), (creed, VBP), (is, VBZ), (a, D...","[(how, WRB), (did, VBD), (apollo, VB), (creed,...","[(apollo, ORG), (creed, ), (is, ), (a, ), (fic...","[(how, ), (did, ), (apollo, ORG), (creed, ), (...","[0.1506184153725464, 0.2875274053249442, 0.057...","[1.0342546228402965, 1.1571023666681808, 1.787..."
3,in the united states the title of federal jud...,how long is the term for federal judges,"[[-1.4634616, -0.11238795, -1.1756282, -0.3831...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[in, the, united, states, the, title, of, fede...","[how, long, is, the, term, for, federal, judges]","[(in, IN), (the, DT), (united, JJ), (states, V...","[(how, WRB), (long, JJ), (is, VBZ), (the, DT),...","[(in, ), (the, GPE), (united, GPE), (states, G...","[(how, ), (long, ), (is, ), (the, ), (term, ),...","[0.08829203392786322, 0.3204713483897852, 0.16...","[0.6464091392751853, 0.7778250162349475, 0.429..."
4,the beretta 21a bobcat is a small pocket sized...,how a beretta model 21 pistols magazines works,"[[-0.23086248, -0.8100985, -0.57038724, -0.738...","[[-0.9877842, 0.7618303, -2.012355, 1.7880102,...","[the, beretta, 21a, bobcat, is, a, small, pock...","[how, a, beretta, model, 21, pistols, magazine...","[(the, DT), (beretta, NN), (21a, CD), (bobcat,...","[(how, WRB), (a, DT), (beretta, NN), (model, N...","[(the, ), (beretta, ), (21a, ), (bobcat, ), (i...","[(how, ), (a, ), (beretta, PRODUCT), (model, )...","[0.21161559146390707, 0.8290244630930195, 0.22...","[0.6464091392751853, 0.4550243894508378, 1.243..."


In [36]:
print(train_doc_ques["Document"].head(1))

0    a partly submerged glacier cave on perito more...
Name: Document, dtype: object


In [37]:
# reduce the dataframe to just tokens and embeddings:
doc_emb_train = train_doc_ques[["Doc_Tokens", "Doc_Embeddings", "Doc_TFIDF"]]
doc_pos_ner = pd.DataFrame({"Doc_POS": train_doc_pos_ohv, "Doc_NER": train_doc_ner_ohv})
doc_emb_train = pd.concat([doc_emb_train, doc_pos_ner], axis=1)

q_emb_train = train_doc_ques[["Q_Tokens", "Q_Embeddings", "Q_TFIDF"]]
q_pos_ner = pd.DataFrame({"Q_POS": train_q_pos_ohv, "Q_NER": train_q_ner_ohv})
q_emb_train = pd.concat([q_emb_train, q_pos_ner], axis=1)

In [38]:
doc_emb_test = test_doc_ques[["Doc_Tokens", "Doc_Embeddings", "Doc_TFIDF"]]
doc_pos_ner = pd.DataFrame({"Doc_POS": test_doc_pos_ohv, "Doc_NER": test_doc_ner_ohv})
doc_emb_test = pd.concat([doc_emb_test, doc_pos_ner], axis=1)

q_emb_test = test_doc_ques[["Q_Tokens", "Q_Embeddings", "Q_TFIDF"]]
q_pos_ner = pd.DataFrame({"Q_POS": test_q_pos_ohv, "Q_NER": test_q_ner_ohv})
q_emb_test = pd.concat([q_emb_test, q_pos_ner], axis=1)

### Word Embeddings (Doc and Qn)

The embeddings of the questions and answers of the train and test set can be found here:

-   Train Document - doc_emb_train
-   Train Q - q_emb_train
-   Test Document - doc_emb_test
-   Test Q - q_emb_test

The max_document size is 1675 and max_question size is 23.


In [39]:
doc_emb_train.loc[0]

Doc_Tokens        [a, partly, submerged, glacier, cave, on, peri...
Doc_Embeddings    [[0.66377705, -0.66919404, -2.8755727, 0.84947...
Doc_TFIDF         [0.24679288919367473, 0.145118630440616, 0.161...
Doc_POS           [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
Doc_NER           [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
Name: 0, dtype: object

In [40]:
# len(mini.iloc[2])

In [41]:
len(doc_emb_train)

2117

In [42]:
def full_array(data, data_type="Document"):
    num_vec_length = 156
    max_doc = 1675
    max_qn = 23
    zero_vec = np.zeros(156)

    if data_type == "Document":
        full_vec = []  # create a list for list of list for document
        for dat in range(len(data)):  # go through each line
            doc_ques = data.loc[dat]  # document data
            v = []  # create list to each word
            for j in range(len(doc_ques.iloc[0])):
                vn = []  # list of concat word embeddings
                vn.append(doc_ques.iloc[1][j].tolist())
                vn.append(doc_ques.iloc[2][j])
                vn.append(doc_ques.iloc[3][j].tolist())
                vn.append(doc_ques.iloc[4][j].tolist())
                flatten = [
                    item
                    for sublist in vn
                    for item in (sublist if isinstance(sublist, list) else [sublist])
                ]
                v.append(flatten)
            while len(v) < max_doc:
                v.append(zero_vec)
            full_vec.append(v)

    if data_type == "Question":
        full_vec = []  # create a list for list of list for document
        for dat in range(len(data)):  # go through each line
            doc_ques = data.loc[dat]  # document data
            v = []  # create list to each word
            for j in range(len(doc_ques.iloc[0])):
                vn = []  # list of concat word embeddings
                vn.append(doc_ques.iloc[1][j].tolist())
                vn.append(doc_ques.iloc[2][j])
                vn.append(doc_ques.iloc[3][j].tolist())
                vn.append(doc_ques.iloc[4][j].tolist())
                flatten = [
                    item
                    for sublist in vn
                    for item in (sublist if isinstance(sublist, list) else [sublist])
                ]
                v.append(flatten)
            while len(v) < max_qn:
                v.append(zero_vec)
            full_vec.append(v)
    return full_vec

In [43]:
# Training/Test Documents to pass in, takes about a min
final_doc_train = full_array(doc_emb_train, data_type="Document")
final_doc_test = full_array(doc_emb_test, data_type="Document")

In [44]:
# Training/Test Questions to pass in, takes about a few seconds
final_qn_train = full_array(q_emb_train, data_type="Question")
final_qn_test = full_array(q_emb_test, data_type="Question")

### Converting into Tensors:


In [79]:
import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [80]:
# takes a min
tf_final_doc_train = torch.tensor(final_doc_train, device=device)
tf_final_doc_test = torch.tensor(final_doc_test, device=device)
tf_final_qn_train = torch.tensor(final_qn_train, device=device)
tf_final_qn_test = torch.tensor(final_qn_test, device=device)

In [81]:
# check dimensions
print(tf_final_doc_train.shape)
print(tf_final_doc_test.shape)
print(tf_final_qn_train.shape)
print(tf_final_qn_test.shape)

torch.Size([2117, 1675, 156])
torch.Size([630, 1675, 156])
torch.Size([2117, 23, 156])
torch.Size([630, 23, 156])


In [82]:
print(tf_final_doc_train[0])

tensor([[ 0.6638, -0.6692, -2.8756,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.5944, -0.6558, -0.0168,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2131,  0.0257, -0.1679,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)


In [83]:
print(final_doc_test[0])

[[1.2212064266204834, 0.0490167997777462, -0.07282713800668716, 0.9991928935050964, 0.11272181570529938, -1.9128384590148926, 2.465834140777588, 0.26472216844558716, 0.1378515660762787, -1.5851463079452515, -1.9296742677688599, -2.815145969390869, 0.002899299608543515, 0.05974029749631882, 0.3342489004135132, -1.0661695003509521, 0.8430624008178711, -1.7231285572052002, -0.8874247670173645, 0.34609130024909973, -0.2936844229698181, 4.154630661010742, -0.2469148486852646, -2.5089404582977295, 1.192534327507019, -1.0775429010391235, 0.9564865827560425, 0.5779495239257812, 0.9680888652801514, -1.1172924041748047, -0.9766153693199158, -1.1416664123535156, 1.1499736309051514, -0.7076790928840637, -0.9265713095664978, 1.0916370153427124, -1.0015441179275513, 1.2436909675598145, -0.2523607909679413, 1.2604258060455322, -2.1118733882904053, 0.42035791277885437, 0.1787702590227127, 0.3301529586315155, -0.025156494230031967, 2.9827792644500732, -0.7889190912246704, -0.028432833030819893, -0.4115

In [84]:
print(1 in tf_final_doc_train)

True


In [85]:
train_doc_ans_labels[:1]

[['N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'S',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'E',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N',
  'N']]

In [86]:
print(train_doc_ques["Document"][0])

a partly submerged glacier cave on perito moreno glacier   the ice facade is approximately 60 m high ice formations in the titlis glacier cave a glacier cave is a cave formed within the ice of a glacier   glacier caves are often called ice caves   but this term is properly used to describe bedrock caves that contain year round ice 


In [87]:
len(train_doc_ans_labels)

2117

In [88]:
label_to_int = {
    "N": 0,
    "S": 1,
    "I": 2,
    "E": 3,
}  # Define a mapping from labels to integers

target = []
for labels in train_doc_ans_labels:
    encoded_labels = [label_to_int[label] for label in labels]
    # encoded_tensor = torch.tensor(encoded_labels, device=device)
    target.append(encoded_labels)

In [89]:
target[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [391]:
start_targets = []
inner_targets = []
end_targets = []
none_targets = []

# define the desired length
desired_length = 1675


for labels in train_doc_ans_labels:
    start_target = [1 if label == "S" else 0 for label in labels]
    inner_target = [1 if label == "I" else 0 for label in labels]
    end_target = [1 if label == "E" else 0 for label in labels]
    none_target = [1 if label == "N" else 0 for label in labels]

    start_target = torch.tensor(start_target, device=device)
    inner_target = torch.tensor(inner_target, device=device)
    end_target = torch.tensor(end_target, device=device)
    none_target = torch.tensor(none_target, device=device)

    start_target_padding = desired_length - start_target.shape[0]
    inner_target_padding = desired_length - inner_target.shape[0]
    end_target_padding = desired_length - end_target.shape[0]
    none_target_padding = desired_length - none_target.shape[0]

    start_target = torch.nn.functional.pad(start_target, (0, start_target_padding))
    inner_target = torch.nn.functional.pad(inner_target, (0, inner_target_padding))
    end_target = torch.nn.functional.pad(end_target, (0, end_target_padding))
    none_target = torch.nn.functional.pad(none_target, (0, none_target_padding))

    start_targets.append(start_target)
    inner_targets.append(inner_target)
    end_targets.append(end_target)
    none_targets.append(none_target)

In [91]:
print(tf_final_doc_train[0].long())

tensor([[ 0,  0, -2,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        ...,
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]])


### Model


### Training


In [92]:
MAX_DOC_LENGTH = 1675  # Max doc length
MAX_QN_LENGTH = 23  # Max question length

In [208]:
import torch
from torch import Tensor
import torch.nn as nn


class DocumentBiRNN(nn.Module):
    def __init__(
        self,
        hidden_size: int,
        input_size: int,
        num_layers=1,
    ):
        super(DocumentBiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, hidden_size)  # Linear projection layer
        self.lstm = nn.LSTM(
            hidden_size,
            hidden_size,
            num_layers=num_layers,
            bidirectional=True,
        )

    def forward(self, input: Tensor):
        input = input.to(self.linear.weight.dtype)
        input_linear = self.linear(input.unsqueeze(1))
        output: Tensor
        output, _ = self.lstm(input_linear)
        print("output shape: ", output.shape)
        return output

In [235]:
import torch
from torch import Tensor
import torch.nn as nn

import torch.nn.functional as F
import torch.nn as nn


class QuestionBiRNN(nn.Module):
    def __init__(
        self,
        hidden_size: int,
        output_size: int,
        max_length: int,
        num_layers=1,
    ):
        super(QuestionBiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length
        self.linear = nn.Linear(output_size, hidden_size)  # Linear projection layer
        self.lstm = nn.LSTM(
            hidden_size,
            hidden_size,
            num_layers=num_layers,
            bidirectional=True,
        )
        self.out = nn.Linear(self.hidden_size * 2, self.output_size)

    def forward(self, input: Tensor):
        input = input.to(self.linear.weight.dtype)
        input_linear = self.linear(input.unsqueeze(1))
        _, (hidden, _) = self.lstm(input_linear)
        # Concatenate the last hidden state from the forward and backward LSTMs for the question
        hidden_forward = hidden[0]
        hidden_backward = hidden[1]
        hidden_concatenated = torch.cat((hidden_forward, hidden_backward), dim=1)

        # Pass the concatenated hidden state through the fully connected layer to get weighted sum
        summary = self.out(hidden_concatenated)
        print("hidden shape: ", hidden.shape)
        print("summary shape: ", summary.shape)
        return summary

In [302]:
class Attention(nn.Module):
    def __init__(self, document_hidden_size, question_hidden_size):
        super(Attention, self).__init__()
        self.projection = nn.Linear(document_hidden_size, question_hidden_size)

    def forward(self, document_output, question_summary):
        # Project the document output to have the same hidden size as the question summary
        document_output = self.projection(document_output)

        # Adjust the dimensions of tensors
        document_output = document_output.transpose(0, 1)  # shape: (1, 1675, 156)
        question_summary = question_summary.unsqueeze(0)  # shape: (1, 1, 156)

        # Compute the dot product between the projected document output and the question summary
        attention_scores = torch.bmm(document_output, question_summary.transpose(1, 2))

        # Apply the softmax function to the attention scores to obtain the attention weights
        attention_weights = nn.functional.softmax(attention_scores, dim=1)

        # Compute the weighted sum of the projected document output using the attention weights
        attention_output = torch.bmm(
            attention_weights.transpose(1, 2), document_output
        ).squeeze(1)

        print("attention output shape: ", attention_output.shape)

        return attention_output

In [375]:
class Attention(nn.Module):
    def __init__(self, document_hidden_size, question_hidden_size):
        super(Attention, self).__init__()
        self.projection = nn.Linear(document_hidden_size, question_hidden_size)

    def forward(self, document_output, question_summary):
        # Save the original document output
        original_document_output = document_output

        # Project the document output to have the same hidden size as the question summary
        document_output = self.projection(document_output)

        # Adjust the dimensions of tensors
        document_output = document_output.transpose(0, 1)  # shape: (1, 1675, 156)
        question_summary = question_summary.unsqueeze(0)  # shape: (1, 1, 156)

        # Compute the dot product between the projected document output and the question summary
        attention_scores = torch.bmm(document_output, question_summary.transpose(1, 2))

        print("attention scores shape: ", attention_scores.shape)

        # Apply the softmax function to the attention scores to obtain the attention weights
        attention_weights = (
            nn.functional.softmax(attention_scores, dim=2).squeeze(0).squeeze(1)
        )

        print("attention weights shape: ", attention_weights.shape)

        return attention_weights

In [289]:
class ReadingComprehensionModel(nn.Module):
    def __init__(self, document_rnn, question_rnn, attention, hidden_size, output_size):
        super(ReadingComprehensionModel, self).__init__()
        self.document_rnn = document_rnn
        self.question_rnn = question_rnn
        self.attention = attention
        self.start_pred = nn.Linear(hidden_size, output_size)
        self.inner_pred = nn.Linear(hidden_size, output_size)
        self.end_pred = nn.Linear(hidden_size, output_size)
        self.none_pred = nn.Linear(hidden_size, output_size)

In [401]:
from torch.utils.data import TensorDataset, DataLoader


def trainIter(
    model,
    document_inputs,
    question_inputs,
    start_targets,
    inner_targets,
    end_targets,
    none_targets,
    num_epochs,
    criterion,
    optimizer,
):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for (
            document_input,
            question_input,
            start_target,
            inner_target,
            end_target,
            none_target,
        ) in zip(
            document_inputs,
            question_inputs,
            start_targets,
            inner_targets,
            end_targets,
            none_targets,
        ):
            optimizer.zero_grad()

            document_output = model.document_rnn(document_input)
            question_summary = model.question_rnn(question_input)

            weighted_doc_output = model.attention(document_output, question_summary)

            start_logits = model.start_pred(weighted_doc_output)
            inner_logits = model.inner_pred(weighted_doc_output)
            end_logits = model.end_pred(weighted_doc_output)
            none_logits = model.none_pred(weighted_doc_output)

            print("start logits shape: ", start_logits.shape)
            print("start target shape: ", start_target.shape)

            print("start_logits: ", start_logits)
            print("start_target: ", start_target)

            # loss = (
            #     criterion(start_logits, start_target.unsqueeze(0))
            #     + criterion(inner_logits, inner_target.unsqueeze(0))
            #     + criterion(end_logits, end_target.unsqueeze(0))
            #     + criterion(none_logits, none_target.unsqueeze(0))
            # )
            start_loss = criterion(start_logits, start_target)
            inner_loss = criterion(inner_logits, inner_target)
            end_loss = criterion(end_logits, end_target)
            none_loss = criterion(none_logits, none_target)

            loss = start_loss + inner_loss + end_loss + none_loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(document_inputs)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

In [98]:
def evaluate(model, eval_data):
    model.eval()
    total_examples = 0
    total_correct = 0

    with torch.no_grad():
        for document_input, question_input, start_target, end_target in eval_data:
            document_output, _ = model.document_rnn(document_input.long())
            _, question_hidden = model.question_rnn(question_input.long())

            weighted_doc_output = model.attention(document_output, question_hidden)

            start_logits = model.start_pred(weighted_doc_output)
            end_logits = model.end_pred(weighted_doc_output)

            start_pred = torch.argmax(start_logits, dim=0)
            end_pred = torch.argmax(end_logits, dim=0)

            total_examples += 1
            if start_pred == start_target and end_pred == end_target:
                total_correct += 1

    accuracy = total_correct / total_examples
    return accuracy

In [137]:
print(tf_final_doc_train.shape)
print(tf_final_doc_train.long().min())
print(tf_final_doc_train.long().max())

torch.Size([2117, 1675, 156])
tensor(-10)
tensor(9)


In [100]:
print(tf_final_doc_train[0])

tensor([[ 0.6638, -0.6692, -2.8756,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.5944, -0.6558, -0.0168,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2131,  0.0257, -0.1679,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)


In [101]:
print(len(start_targets))
print(len(start_targets[0]))

2117
59


In [102]:
# test_hidden_size = 128
# test_learning_rate = 0.1

# test_document_num_embeddings = tf_final_doc_train.shape[2]
# test_question_num_embeddings = tf_final_qn_train.shape[2]
# test_document_embedding = nn.Embedding(
#     test_document_num_embeddings, test_hidden_size
# ).cpu()
# test_embeds = test_document_embedding(tf_final_doc_train.cpu().long()[0]).cpu()
# print(test_embeds.shape)

In [103]:
# dataset = TensorDataset(
#     tf_final_doc_train,
#     tf_final_qn_train,
#     start_targets,
#     inner_targets,
#     end_targets,
#     none_targets,
# )

# # Define batch size and create DataLoader
# batch_size = 32
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
# for

In [402]:
from torch import optim


# def train():
hidden_size = 128
hidden_size_attn_doc = 256
hidden_size_attn_ques = 156
learning_rate = 0.1

document_num_embeddings = tf_final_doc_train.shape[2]
question_num_embeddings = tf_final_qn_train.shape[2]
doc_seq_length = tf_final_doc_train.shape[1]
# document_embedding = nn.Embedding(document_num_embeddings, hidden_size).to(device)
# question_embedding = nn.Embedding(question_num_embeddings, hidden_size).to(device)


document_rnn = DocumentBiRNN(
    hidden_size=hidden_size, input_size=document_num_embeddings
).to(device)
question_rnn = QuestionBiRNN(
    hidden_size=hidden_size,
    output_size=question_num_embeddings,
    max_length=MAX_QN_LENGTH,
).to(device)
attention = Attention(
    document_hidden_size=hidden_size_attn_doc,
    question_hidden_size=hidden_size_attn_ques,
)
reading_comp = ReadingComprehensionModel(
    document_rnn,
    question_rnn,
    attention,
    hidden_size=doc_seq_length,
    output_size=doc_seq_length,
)
optimizer = optim.Adam(reading_comp.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# # Create TensorDatasets from your data tensors
# dataset = TensorDataset(tf_final_doc_train, tf_final_qn_train)

# # Define batch size and create DataLoader
# batch_size = 32
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

trainIter(
    reading_comp,
    tf_final_doc_train,
    tf_final_qn_train,
    start_targets,
    inner_targets,
    end_targets,
    none_targets,
    10,
    criterion,
    optimizer,
)


# train()

output shape:  torch.Size([1675, 1, 256])
hidden shape:  torch.Size([2, 1, 128])
summary shape:  torch.Size([1, 156])
attention scores shape:  torch.Size([1, 1675, 1])
attention weights shape:  torch.Size([1675])
start logits shape:  torch.Size([1675])
start target shape:  torch.Size([1675])
start_logits:  tensor([-0.5791,  0.0125,  0.4914,  ..., -0.7150,  0.4178,  0.8422],
       grad_fn=<AddBackward0>)
start_target:  tensor([0, 0, 0,  ..., 0, 0, 0])


RuntimeError: Expected floating point type for target with class probabilities, got Long

In [393]:
for tokens in start_targets:
    print(len(tokens))

1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675
1675


In [397]:
print(start_targets[0][:59])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
